Resumen de a clase anterior
========================

Objetivos:
* Breve introducción al metabolismo celular
* Simplificaciones de su funcionamiento
* Modelos matemáticos
* Implementación computacional en Python usando programación lineal.


Breve introducción al metabolismo celular
-----------------------------------------

¿Por qué es importante estudiar el metabolismo celular?

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_quimiostato.jpeg" alt="Drawing" style="width: 300px;"/>

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_dinamica.jpeg" alt="Drawing" style="width: 300px;"/>


Si tenemos un modelo del metabolismo podemos anticipar lo que sucederá bajo nuevas condiciones, lo que nos ayudará a diseñar procesos más eficientes. 

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_cajaNegra.png" alt="Drawing" style="width: 300px;"/>




El problema es que incluso la red de un organismo simple, como el de la bacteria *Escherichia coli*, luce así:

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_eColiMetabolis.png" alt="Drawing" style="width: 400px;"/>

["Fuente KEGG"](https://www.genome.jp/kegg-bin/show_pathway?eco01100+M00125)

Resumen: ¡Es complicado!. **Busquemos un modelo a escala más pequeño para ganar intución.**


Análisis en estado estacionario de una red simple
=========================================

Supongamos que este metabolismo primigenio está dado por la siguiente red (inventada) de reacciones:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="400">


1. ¿Cuál es el balance de masa de sus metabolitos en estado estacionario?
2. ¿Cómo podemos representar el balance de masa en notación matricial?
3. ¿Cómo podemos calcular el valor de los flujos metabólicos?

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_matriz.jpg" width="500">

Clase 5
=======

Objetivos (continuación de la Clase 4):
* Modelos matemáticos
* Implementación computacional en Python usando programación lineal.

¿Cómo estimar la configuración de flujos metabólicos cuando no hay suficientes ecuaciones?
-----------------------------------------

Incluso una red simple (la de abajo es la más simple que puedo imaginar) adolece de suficientes ecuaciones para determinar la configuración de flujos metabólicos.

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase5_solucionesAlternativas.png" width="500">

Propuesta: Dentro de todas las soluciones alternativas escojamos aquella que sea maximize una función biológica
---------

La función biológica más utilizada es la reacción de produción biomasa. Esta hipotesis se basa en la suposición de que a través de la evolución los microorganismos han desarrollado mecanismos de regulación de flujos que resultan en distribuciones de flujos que maximizan la velocidad de producción de biomasa. 

¿Cómo se construye una reacción de producción de biomasa?

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase5_precursors.jpg" width="500">

[Imagen del artículo "The origin of life", Scientific American](https://www.americanscientist.org/article/the-origin-of-life)

Otros función biológica usada es la maximizacion de la producción de ATP.


Usemos como ejemplo nuestra red simple


Qué forma tiene el vector `c^T` si queremos:

1. Maximizar la producción de metano (CH4)
2. Maximizar la producción de la molecula de alta energía (F420)


Formulación matemática de un problema de optimización (lineal)
------------

La distribución de flujos metabólicos de una celula puede ser modelado como:

\begin{align}
\mbox{max}\ & f(x)=c^Tv \\
\mbox{s.a.} & \\
&Sv=0 \\
&LB<=v<=UB
\end{align}

En donde `S` es la matriz estequiométrica, `v` los flujos metabólicos, y `LB` y `UB` que son los limites inferiores (lower bound) y superiores (upper bound) de cada flujo. Por otra parte, `c` contiene los coeficientes que ponderan el aporte de cada `v` a la generación de una función apropiada, las cuales tipicamente corresponden a biomasa o producción de energía. 

Este tipo de modelos en la literatura se conocen como Contraint-based models (modelos en base a restricciones). (Aquí un review interesante)(https://www.nature.com/articles/nrmicro2737)



Programación lineal en Python
=========================


Para resolver este tipo de problemas usando el método simplex existe la librería `scipy.optimize.linprog`. Típicamente un modelo metabólico consiste en restricciones de igualdad, por ejemplo: 


\begin{align}
\mbox{min}\ &f(x)=70x_1+80X_2+85X_3 \\
\mbox{s.a:}\ & \\
&x_1+x_2+x_3+x_4=999 \\
&x_1+4x_2+8x_3+x_5 = 4500 \\
&40x_1 + 30x_2 +20x_3 = 36000 \\
&3x_1 + 2x_2 +4x_3 +x_6= 2700 \\
&x>=0
\end{align}

puede ser resuelto con la siguiente sintaxis:

``` python
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

A = np.array([
[1, 1, 1, 0, 0, 0],
[1, 4, 8, 1, 0, 0],
[40, 30, 20, 0, 1, 0],
[3, 2, 4, 0, 0, 1]])

b = np.array([999, 4500, 36000, 2700])
c = np.array([70, 80, 85, 0, 0, 0])

LB=[0]*6      # [0,0,0,0,0,0]
UB=[100000]*6 # ya sabes la idea

res = linprog(c, A_eq=A, b_eq=b, bounds=list(zip(LB,UB)))
print('Optimal value:', res.fun, '\nX:', res.x)
```
Sin embargo, `linprog` tambien nos permite resolver problemas en donde las restricciones son una mezcla de igualdades y desigualdades. Por ejemplo:

\begin{align}
\mbox{min}\ &f(x)=70x_1+80X_2+85X_3 \\
\mbox{s.a:}\ & \\
&x_1+x_2+x_3=999 \\
&x_1+4x_2+8x_3 \le 4500 \\
&40x_1 + 30x_2 +20x_3 \le 36000 \\
&3x_1 + 2x_2 +4x_3 \le 2700 \\
&x>=0
\end{align}

Puede ser resuelto con las siguiente sintaxis:

``` python
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

A_eq = np.array([[1,1,1]])
b_eq = np.array([999])

A_ub = np.array([
[1, 4, 8],
[40,30,20],
[3,2,4]])

b_ub = np.array([4500, 36000,2700])

c = np.array([70, 80, 85])

LB=[0]*3
UB=[100000]*3

res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub,bounds=list(zip(LB,UB)) )
print('Optimal value:', res.fun, '\nX:', res.x)
```

Ejercicio
-------------

Calcular el valor de los flujos metabólicos usando numpy.

1. Asumiendo que el flujo limitante es E1=10 [mmol/gDW/h] y que el organismo maximiza la producción de CH4.
2. Igual que 1 pero maximizando la producción de F420.